In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
import tensorflow as tf

import json
from pathlib import Path

/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_to_training = Path("training")
path_to_test = Path("test")

In [3]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<[^>]*>|(?:um|uh)', '', text)

    # Tokenization des mots
    words = word_tokenize(text)

    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Stemming (réduction à la racine des mots)
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words]

    # Rejoindre les mots traités en une seule chaîne de texte
    processed_text = ' '.join(words)

    return processed_text


def get_xi(transcription_id, path_to_data = path_to_training):
       discourse_graph = [] # list, i attribute j
       with open(path_to_data / f"{transcription_id}.txt", 'r') as f:
              for line in f: discourse_graph.append(line.strip())
       
       with open(path_to_data / f"{transcription_id}.json", 'r') as f:
              transcription = json.load(f)

       x_i = [] # list, speaker: attribute: text
       prefix_i = ["" for i in range(len(transcription))] # list des prelations pour lesquelles i est prefixes
       suffix_j = ["" for i in range(len(transcription))] # list des prelations pour lesquelles j est suffixes

       for line in discourse_graph:
              tmp = line.split()
              i = int(tmp[0])
              j = int(tmp[-1])

              if prefix_i[i]!= "": prefix_i[i]+=','
              prefix_i[i] += ("p"+tmp[1])

              if suffix_j[j]!= "": suffix_j[j]+=','
              suffix_j[j] += ("s"+tmp[1])

       for i in range(len(transcription)):
              replique = transcription[i]
              text = preprocess_text(replique['text'])
              x_i.append(text)

       return x_i, prefix_i, suffix_j


In [4]:
# Recup des ids
transcription_ids = []

transcripts = path_to_training.glob('*.json')
for transcript in transcripts:
    transcription_ids.append(transcript.name[:-5])

In [5]:
# creation de X contenant les repliques x_i
X = [] #list des repliques
A = [] #list des prefixes
B = [] #list des suffixes

for transcription_id in  transcription_ids:
    x_i, prefix_i, suffix_j = get_xi(transcription_id)
    X.extend(x_i)
    A.extend(prefix_i)
    B.extend(suffix_j)

In [6]:
Z = np.concatenate([np.array(X).reshape(-1,1), np.array(A).reshape(-1,1), np.array(B).reshape(-1,1)], axis=1)
Z.shape

(72623, 3)

In [7]:
# creation de y contenant les labels pour chaque x_i
y = [] # concatenation des labels
with open("training_labels.json", 'r') as f:
    labels = json.load(f)

for transcription_id in transcription_ids:
    y.extend(labels[transcription_id])

y = np.array(y).reshape(-1,1)

print(y.shape)

(72623, 1)


### Séparation en train_test_spli

In [8]:
Z_train, Z_valid, y_train, y_valid = train_test_split(Z, y, test_size=0.3, random_state=42)

### Recup de A_train, B_train et transformation en variables dummies

In [9]:
X_train = Z_train[:,0]
A_train = Z_train[:,1]
B_train = Z_train[:,2]

A_train = pd.DataFrame(A_train, columns=['variable'])
A_train = A_train['variable'].str.get_dummies(sep=',')

B_train = pd.DataFrame(B_train, columns=['variable'])
B_train = B_train['variable'].str.get_dummies(sep=',')

A_train_cols = A_train.columns
B_train_cols = B_train.columns

A_train = A_train.values
B_train = B_train.values

## Vectorisation de X_train et Reconstruction de Z_train = X_train + A_train + B_train

In [10]:
bert = SentenceTransformer('distilbert-base-uncased')
X_train_bert = bert.encode(X_train, show_progress_bar=True)

Z_train = np.concatenate([X_train_bert, A_train, B_train], axis=1)

No sentence-transformers model found with name /Users/dabereabasse/.cache/torch/sentence_transformers/distilbert-base-uncased. Creating a new one with MEAN pooling.
Batches: 100%|██████████| 1589/1589 [01:44<00:00, 15.13it/s]


### Recup de A_valid, B_valid et transformation en variables dummies avec les memes colonnes que A_train, B_train

In [11]:
X_valid = Z_valid[:,0]
A_valid = Z_valid[:,1]
B_valid = Z_valid[:,2]

# variables dummies
A_valid = pd.DataFrame(A_valid, columns=['variable'])
A_valid = A_valid['variable'].str.get_dummies(sep=',')

B_valid = pd.DataFrame(B_valid, columns=['variable'])
B_valid = B_valid['variable'].str.get_dummies(sep=',')

# rearangement des colonnes de A_valid et B_valid pour que ça soit pareil avec celles des train

A_valid = A_valid.reindex(columns=A_train_cols).fillna(0)
B_valid = B_valid.reindex(columns=B_train_cols).fillna(0)

A_valid = A_valid.values
B_valid = B_valid.values

In [12]:
X_valid_bert = bert.encode(X_valid, show_progress_bar=True)
Z_valid = np.concatenate([X_valid_bert, A_valid, B_valid], axis=1)

Batches: 100%|██████████| 681/681 [00:44<00:00, 15.15it/s]


## DNN

In [15]:
count_class_0 = len(y_train) - np.sum(y_train)
count_class_1 = np.sum(y_train)

total = len(y_train)

frequency_class_0 = count_class_0 / total
frequency_class_1 = count_class_1 / total

inverse_weight_class_0 = 1 / frequency_class_0
inverse_weight_class_1 = 1 / frequency_class_1

class_weights = {0: inverse_weight_class_0, 1: inverse_weight_class_1}

In [17]:
architectures = [
    [700, 600, 500, 400, 300, 200, 100, 50],
    [650, 500, 350, 200, 50],
    [700, 600, 500, 400, 300, 200, 100],
    [600, 400, 200],
    [500, 300, 100],
    [400, 200, 100, 50],
    [700, 600, 500, 400, 300],
    [600, 500, 400, 300, 200, 100, 50],
    [700, 600, 500, 400],
    [600, 400, 200, 100]
]

In [18]:
# architectures = [[600, 500, 300, 100, 50], [600, 500, 300, 100], [500, 300, 100], [300, 100, 50], [400, 250, 150, 100, 50], [400, 250, 150, 100]]
scores = []
for i in range(len(architectures)):
    model = Sequential()
    model.add(Dense(architectures[i][0], input_dim=800, activation='relu'))
    model.add(Dropout(0.5))
    for j in range(1, len(architectures[i])):
        model.add(Dense(architectures[i][j], activation='relu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model_history = model.fit(Z_train, y_train, epochs=20, batch_size=32, validation_data=(Z_valid, y_valid), class_weight=class_weights)
    y_pred_dnn = model.predict(Z_valid)
    y_pred = np.where(y_pred_dnn>=0.5, 1, 0)
    scores.append(f1_score(y_valid, y_pred))

Epoch 1/20
1589/1589 [==============================] - 202s 127ms/step - loss: 1.1033 - accuracy: 0.6634 - val_loss: 0.6068 - val_accuracy: 0.6387
Epoch 2/20
1589/1589 [==============================] - 15s 9ms/step - loss: 1.0034 - accuracy: 0.6863 - val_loss: 0.4830 - val_accuracy: 0.6984
Epoch 3/20
1589/1589 [==============================] - 51s 32ms/step - loss: 1.0185 - accuracy: 0.6905 - val_loss: 0.4646 - val_accuracy: 0.6721
Epoch 4/20
1589/1589 [==============================] - 124s 78ms/step - loss: 0.9899 - accuracy: 0.7160 - val_loss: 0.4321 - val_accuracy: 0.7689
Epoch 5/20
1589/1589 [==============================] - 8s 5ms/step - loss: 1.0103 - accuracy: 0.7510 - val_loss: 0.4268 - val_accuracy: 0.7570
Epoch 6/20
1589/1589 [==============================] - 8s 5ms/step - loss: 1.0419 - accuracy: 0.7333 - val_loss: 0.4549 - val_accuracy: 0.7150
Epoch 7/20
1589/1589 [==============================] - 8s 5ms/step - loss: 1.0331 - accuracy: 0.7281 - val_loss: 0.4806 - val

In [19]:
trasholds = np.linspace(0.4, 0.7, 100)
arch_selected = [architectures[i] for i in range(len(architectures)) if scores[i]>=0.57]

print("architectures selected:", len(arch_selected))

scores = [0 for i in range(len(arch_selected))]
best_trasholds = [0 for i in range(len(arch_selected))]
best_p = [[] for i in range(len(arch_selected))]

for i in range(len(arch_selected)):
    print(i + 1, "/", len(arch_selected))
    arch = arch_selected[i]
    for k in range(5):
        p = np.random.random(len(arch))

        model = Sequential()
        model.add(Dense(arch[0], input_dim=800, activation='relu'))
        model.add(Dropout(p[0]))
        for j in range(1, len(arch)):
            model.add(Dense(arch[j], activation='relu'))
            model.add(Dropout(p[j]))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(Z_train, y_train, epochs=20, batch_size=32, validation_data=(Z_valid, y_valid), class_weight=class_weights)

        y_pred_dnn = model.predict(Z_valid)
        for trashold in trasholds:
            y_pred = np.where(y_pred_dnn>=trashold, 1, 0)
            fscore = f1_score(y_valid, y_pred)
            if fscore > scores[i]:
                scores[i] = fscore
                best_trasholds[i] = trashold
                best_p[i] = p

architectures selected: 2
1 / 2


Epoch 1/20
1589/1589 [==============================] - 3s 2ms/step - loss: 1.1287 - accuracy: 0.6471 - val_loss: 0.5132 - val_accuracy: 0.6544
Epoch 2/20
1589/1589 [==============================] - 2s 1ms/step - loss: 1.0080 - accuracy: 0.6808 - val_loss: 0.4925 - val_accuracy: 0.6850
Epoch 3/20
1589/1589 [==============================] - 2s 2ms/step - loss: 0.9846 - accuracy: 0.6998 - val_loss: 0.5479 - val_accuracy: 0.6648
Epoch 4/20
1589/1589 [==============================] - 2s 2ms/step - loss: 0.9774 - accuracy: 0.7054 - val_loss: 0.4844 - val_accuracy: 0.6924
Epoch 5/20
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9648 - accuracy: 0.7048 - val_loss: 0.5087 - val_accuracy: 0.6693
Epoch 6/20
1589/1589 [==============================] - 2s 2ms/step - loss: 0.9785 - accuracy: 0.7016 - val_loss: 0.4615 - val_accuracy: 0.7193
Epoch 7/20
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9654 - accuracy: 0.7082 - val_loss: 0.4444 - val_accuracy:

In [21]:
for i in range(len(arch_selected)):
    print("architecture:", [800] + arch_selected[i] + [1])
    print("p:", best_p[i])
    print("trashold:", best_trasholds[i])
    print("f1_score:", scores[i])
    print()

architecture: [800, 400, 200, 100, 50, 1]
p: [0.47809168 0.5949902  0.88866298 0.28404015]
trashold: 0.6515151515151515
f1_score: 0.5885714285714285

architecture: [800, 600, 400, 200, 100, 1]
p: [0.77116783 0.75136918 0.74954189 0.10230484]
trashold: 0.5818181818181818
f1_score: 0.5839583333333332



### Architecture 1

In [20]:
Z_train.shape

(50836, 800)

In [21]:
Z_valid.shape

(21787, 800)

In [22]:
Z = np.concatenate([Z_train, Z_valid], axis=0)
Z.shape

(72623, 800)

In [23]:
y_train.shape

(50836, 1)

In [24]:
y_valid.shape

(21787, 1)

In [25]:
y = np.concatenate([y_train, y_valid], axis=0)
y.shape

(72623, 1)

In [40]:
arch = [400, 200, 100]
p = [0.16496388, 0.49430627, 0.37350436]
trashold = 0.6696969696969697
# f1_score = 0.5905423347972628

In [41]:
model = Sequential([
    Dense(arch[0], input_dim=800, activation='relu'),
    Dropout(p[0]),
    Dense(arch[1], activation='relu'),
    Dropout(p[1]),
    Dense(arch[2], activation='relu'),
    Dropout(p[2]),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=30, batch_size=32, class_weight=class_weights)

#y_pred_dnn = model.predict(Z_valid)
#y_pred = np.where(y_pred_dnn>=trashold, 1, 0)

Epoch 1/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.9288 - accuracy: 0.7262
Epoch 2/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.9036 - accuracy: 0.7301
Epoch 3/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8884 - accuracy: 0.7372
Epoch 4/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8810 - accuracy: 0.7386
Epoch 5/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8754 - accuracy: 0.7415
Epoch 6/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8718 - accuracy: 0.7458
Epoch 7/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8645 - accuracy: 0.7486
Epoch 8/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8622 - accuracy: 0.7509
Epoch 9/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.8525 - accuracy: 0.7538
Epoch 10/30
1589/1589 [==============================] - 2s 1ms/step - loss: 0.849

In [42]:
y_pred_dnn = model.predict(Z_valid)
y_pred = np.where(y_pred_dnn>=trashold, 1, 0)

f1_score(y_valid, y_pred)

681/681 [==============================] - 0s 484us/step


0.566790450928382

## Architecture 2

In [152]:
arch = [600, 500, 300, 100, 50]
p = [0.0361852, 0.81013518, 0.35691142, 0.34738723, 0.55398298]
trashold = 0.6303030303030303
# f1_score = 0.5922241396812506

In [156]:
model = Sequential([
    Dense(arch[0], input_dim=800, activation='relu'),
    Dropout(p[0]),
    Dense(arch[1], activation='relu'),
    Dropout(p[1]),
    Dense(arch[2], activation='relu'),
    Dropout(p[2]),
    Dense(arch[3], activation='relu'),
    Dropout(p[3]),
    Dense(arch[4], activation='relu'),
    Dropout(p[4]),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=10, batch_size=32, validation_data=(Z_valid, y_valid), class_weight=class_weights)

y_pred_dnn = model.predict(Z_valid)
y_pred = np.where(y_pred_dnn>=trashold, 1, 0)

Epoch 1/10
1589/1589 [==============================] - 5s 3ms/step - loss: 1.0032 - accuracy: 0.6966 - val_loss: 0.4998 - val_accuracy: 0.6665
Epoch 2/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9423 - accuracy: 0.7140 - val_loss: 0.4775 - val_accuracy: 0.7113
Epoch 3/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9298 - accuracy: 0.7228 - val_loss: 0.4180 - val_accuracy: 0.7594
Epoch 4/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9165 - accuracy: 0.7316 - val_loss: 0.4094 - val_accuracy: 0.7491
Epoch 5/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9104 - accuracy: 0.7401 - val_loss: 0.4095 - val_accuracy: 0.7689
Epoch 6/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9041 - accuracy: 0.7430 - val_loss: 0.4422 - val_accuracy: 0.7179
Epoch 7/10
1589/1589 [==============================] - 4s 3ms/step - loss: 0.9081 - accuracy: 0.7360 - val_loss: 0.4278 - val_accuracy:

In [157]:
f1_score(y_valid, y_pred)

0.5825428763054519

### Architecture 3

In [170]:
arch = [550, 350, 150]
p = [0.09629556, 0.41359307, 0.688347]
trashold = 0.6363636363636364
# f1_score = 0.5933808454554329

In [171]:
model = Sequential([
    Dense(arch[0], input_dim=800, activation='relu'),
    Dropout(p[0]),
    Dense(arch[1], activation='relu'),
    Dropout(p[1]),
    Dense(arch[2], activation='relu'),
    Dropout(p[2]),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=10, batch_size=32, validation_data=(Z_valid, y_valid), class_weight=class_weights)

y_pred_dnn = model.predict(Z_valid)
y_pred = np.where(y_pred_dnn>=trashold, 1, 0)

Epoch 1/10
1589/1589 [==============================] - 4s 2ms/step - loss: 0.9458 - accuracy: 0.7246 - val_loss: 0.4064 - val_accuracy: 0.7520
Epoch 2/10
1589/1589 [==============================] - 4s 2ms/step - loss: 0.9111 - accuracy: 0.7288 - val_loss: 0.4627 - val_accuracy: 0.7583
Epoch 3/10
1589/1589 [==============================] - 4s 2ms/step - loss: 0.8971 - accuracy: 0.7379 - val_loss: 0.4568 - val_accuracy: 0.7199
Epoch 4/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.8956 - accuracy: 0.7378 - val_loss: 0.4679 - val_accuracy: 0.7533
Epoch 5/10
1589/1589 [==============================] - 4s 2ms/step - loss: 0.8822 - accuracy: 0.7439 - val_loss: 0.4747 - val_accuracy: 0.7145
Epoch 6/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.8797 - accuracy: 0.7426 - val_loss: 0.4555 - val_accuracy: 0.7277
Epoch 7/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.8756 - accuracy: 0.7456 - val_loss: 0.4432 - val_accuracy:

In [172]:
f1_score(y_valid, y_pred)

0.5861873226111636

### Architecture 4

In [192]:
arch = [500, 300, 100, 50]
p = [0.46509202, 0.04129562, 0.41971291, 0.7496419 ]
trashold = 0.6818181818181818
# f1_score = 0.5915606703008278

In [193]:
model = Sequential([
    Dense(arch[0], input_dim=800, activation='relu'),
    Dropout(p[0]),
    Dense(arch[1], activation='relu'),
    Dropout(p[1]),
    Dense(arch[2], activation='relu'),
    Dropout(p[2]),
    Dense(arch[3], activation='relu'),
    Dropout(p[3]),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=10, batch_size=32, validation_data=(Z_valid, y_valid), class_weight=class_weights)

y_pred_dnn = model.predict(Z_valid)
y_pred = np.where(y_pred_dnn>=trashold, 1, 0)

Epoch 1/10
1589/1589 [==============================] - 4s 2ms/step - loss: 1.0158 - accuracy: 0.6985 - val_loss: 0.4814 - val_accuracy: 0.7025
Epoch 2/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9613 - accuracy: 0.7016 - val_loss: 0.4770 - val_accuracy: 0.6906
Epoch 3/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9530 - accuracy: 0.7097 - val_loss: 0.4767 - val_accuracy: 0.7039
Epoch 4/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9466 - accuracy: 0.7171 - val_loss: 0.5007 - val_accuracy: 0.6998
Epoch 5/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9400 - accuracy: 0.7210 - val_loss: 0.4251 - val_accuracy: 0.7508
Epoch 6/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9285 - accuracy: 0.7245 - val_loss: 0.4723 - val_accuracy: 0.7123
Epoch 7/10
1589/1589 [==============================] - 3s 2ms/step - loss: 0.9222 - accuracy: 0.7230 - val_loss: 0.4970 - val_accuracy:

In [194]:
f1_score(y_valid, y_pred)

0.5879974462651628

## Prediction sur le test

In [36]:
# Recup des ids
transcription_ids = []

transcripts = path_to_test.glob('*.json')
for transcript in transcripts:
    transcription_ids.append(transcript.name[:-5])

In [37]:
test_labels_model4 = {}

for transcription_id in transcription_ids:
    X, A, B = get_xi(transcription_id, path_to_test)
    Z_test = np.concatenate([np.array(X).reshape(-1,1), np.array(A).reshape(-1,1), np.array(B).reshape(-1,1)], axis=1)

    X_test = Z_test[:,0]
    A_test = Z_test[:,1]
    B_test = Z_test[:,2]

    # variables dummies
    A_test = pd.DataFrame(A_test, columns=['variable'])
    A_test = A_test['variable'].str.get_dummies(sep=',')

    B_test = pd.DataFrame(B_test, columns=['variable'])
    B_test = B_test['variable'].str.get_dummies(sep=',')

    # rearangement des colonnes de A_valid et B_valid pour que ça soit pareil avec celles des train

    A_test = A_test.reindex(columns=A_train_cols).fillna(0)
    B_test = B_test.reindex(columns=B_train_cols).fillna(0)

    A_test = A_test.values
    B_test = B_test.values

    X_test_bert = bert.encode(X_test, show_progress_bar=True)
    Z_test = np.concatenate([X_test_bert, A_test, B_test], axis=1)

    test_pred = model.predict(Z_test)
    test_pred = np.where(test_pred>=trashold, 1, 0)

    test_labels_model4[transcription_id] = test_pred.reshape(-1,).tolist()

Batches: 100%|██████████| 20/20 [00:01<00:00, 16.29it/s]


20/20 [==============================] - 0s 569us/step


Batches: 100%|██████████| 17/17 [00:01<00:00, 12.94it/s]

17/17 [==============================] - 0s 645us/step



Batches: 100%|██████████| 20/20 [00:01<00:00, 12.07it/s]

...

37/37 [==============================] - 0s 603us/step



Batches: 100%|██████████| 24/24 [00:01<00:00, 13.17it/s]

24/24 [==============================] - 0s 534us/step



Batches: 100%|██████████| 46/46 [00:02<00:00, 16.40it/s]

46/46 [==============================] - 0s 522us/step


## make_submission

In [38]:
def make_submission(test_labels, filename= "submission"):
    file = open(filename+".csv", "w")
    file.write("id,target_feature\n")
    for key, value in test_labels.items():
        u_id = [key + "_" + str(i) for i in range(len(value))]
        target = map(str, value) 
        for row in zip(u_id, target):
            file.write(",".join(row))
            file.write("\n")
    file.close()

In [39]:
make_submission(test_labels_model4, "submission_model4_arch_final")